# Keras Tuner  
Quelle: [tensoflow.org](https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=en)

In [11]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt

In [12]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [13]:
img_train = img_train.astype("float32") / 255.0
img_test = img_test.astype("float32") / 255.0

In [14]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation="relu"))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
    
    return model

In [15]:
tuner = kt.Hyperband(model_builder, objective="val_accuracy", max_epochs=10, factor=3, directory="my_dir", project_name="intro_to_kt")

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [16]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

In [17]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"The hyperparameter search is complete. The optimal number of units in the first dense layer is {best_hps.get('units')} and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.")

Trial 4 Complete [00h 00m 15s]
val_accuracy: 0.875249981880188

Best val_accuracy So Far: 0.8868333101272583
Total elapsed time: 00h 01m 01s
INFO:tensorflow:Oracle triggered exit
The hyperparameter search is complete. The optimal number of units in the first dense layer is 416 and the optimal learning rate for the optimizer is 0.001.


In [18]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history["val_accuracy"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1

print("Best epoch: %d" % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6262 - accuracy: 0.7788 - val_loss: 0.3970 - val_accuracy: 0.8534
Epoch 2/50
1500/1500 [==============================] - 1s 998us/step - loss: 0.3826 - accuracy: 0.8608 - val_loss: 0.3732 - val_accuracy: 0.8628
Epoch 3/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3323 - accuracy: 0.8784 - val_loss: 0.3538 - val_accuracy: 0.8723
Epoch 4/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3060 - accuracy: 0.8870 - val_loss: 0.3295 - val_accuracy: 0.8810
Epoch 5/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2843 - accuracy: 0.8940 - val_loss: 0.3508 - val_accuracy: 0.8786
Epoch 6/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2679 - accuracy: 0.9003 - val_loss: 0.3185 - val_accuracy: 0.8875
Epoch 7/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2576 - accuracy: 0.9033 - val_loss: 0.3056 - val_accurac

In [19]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(img_test, label_test, epochs=best_epoch)

Epoch 1/29
313/313 [==============================] - 0s 1ms/step - loss: 0.8876 - accuracy: 0.6936
Epoch 2/29
313/313 [==============================] - 0s 1ms/step - loss: 0.5085 - accuracy: 0.8156
Epoch 3/29
313/313 [==============================] - 0s 1ms/step - loss: 0.4498 - accuracy: 0.8402
Epoch 4/29
313/313 [==============================] - 0s 1ms/step - loss: 0.4137 - accuracy: 0.8492
Epoch 5/29
313/313 [==============================] - 0s 1ms/step - loss: 0.3780 - accuracy: 0.8608
Epoch 6/29
313/313 [==============================] - 0s 1ms/step - loss: 0.3446 - accuracy: 0.8734
Epoch 7/29
313/313 [==============================] - 0s 1ms/step - loss: 0.3560 - accuracy: 0.8688
Epoch 8/29
313/313 [==============================] - 0s 1ms/step - loss: 0.3118 - accuracy: 0.8878
Epoch 9/29
313/313 [==============================] - 0s 1ms/step - loss: 0.2823 - accuracy: 0.8968
Epoch 10/29
313/313 [==============================] - 0s 1ms/step - loss: 0.2823 - accuracy: 0.8938

In [20]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]: ", eval_result)

313/313 [==============================] - 0s 882us/step - loss: 0.1451 - accuracy: 0.9453
[test loss, test accuracy]:  [0.14513497054576874, 0.9452999830245972]
